In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1sAXfYLXIxZgMrC44LBqDgfYImThZ_kud

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/UniverseNet/configs/gflv2/gflv2_custom.py')

# 처음부터 학습하는 경우
cfg.load_from = 'gfocal_r2n101_dcn_fpn_ms2x.pth' # pretrained

# 이어서 학습하는 경우
# cfg.runner.max_epochs = 32
# cfg.resume_from = cfg.work_dir + '/epoch_24.pth'

print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 480), (1333, 960)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dic

In [3]:
model = build_detector(cfg.model)

2021-05-19 05:20:09,330 - mmdet - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2021-05-19 05:20:09,331 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-19 05:20:09,561 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer3.0.convs.0.conv_offset.weight, layer3.0.convs.0.conv_offset.bias, layer3.0.convs.1.conv_offset.weight, layer3.0.convs.1.conv_offset.bias, layer3.0.convs.2.conv_offset.weight, layer3.0.convs.2.conv_offset.bias, layer3.1.convs.0.conv_offset.weight, layer3.1.convs.0.conv_offset.bias, layer3.1.convs.1.conv_offset.weight, layer3.1.convs.1.conv_offset.bias, layer3.1.convs.2.conv_offset.weight, layer3.1.convs.2.conv_offset.bias, layer3.2.convs.0.conv_offset.weight, layer3.2.convs.0.conv_offset.bias, layer3.2.convs.1.conv_offset.weight, layer3.2.convs.1.conv_offset.bias, layer3.2.convs.2.conv_offset.weight, layer3.2.convs.2.conv_offset.bia

In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.76s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)




loading annotations into memory...


2021-05-19 05:20:19,525 - mmdet - INFO - load checkpoint from gfocal_r2n101_dcn_fpn_ms2x.pth
2021-05-19 05:20:19,526 - mmdet - INFO - Use load_from_local loader


Done (t=0.90s)
creating index...
index created!


2021-05-19 05:20:20,456 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.gfl_cls.weight: copying a param with shape torch.Size([81, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([12, 256, 3, 3]).
size mismatch for bbox_head.gfl_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
2021-05-19 05:20:20,459 - mmdet - INFO - Start running, host: root@03064abcf195, work_dir: /opt/ml/code/UniverseNet/work_dirs/gflv2_r2_101_fpn_dcn_mstrain_2x_aug_trainall
2021-05-19 05:20:20,460 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs
/opt/ml/code/UniverseNet/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
2021-05-19 05:23:09,749 - mmdet - INFO - Epoch [1][100/409]	lr: 1.988e-03, eta: 4:33:51, time: 1.691, data_time: 0.044, memory: 21371, loss

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 11.2 task/s, elapsed: 59s, ETA:     0s

...

2021-05-19 07:11:59,402 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=1.29s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.24s).
Accumulating evaluation results...


2021-05-19 07:12:09,699 - mmdet - INFO - Now best checkpoint is epoch_9.pth.Best bbox_mAP is 0.4460


DONE (t=1.64s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.446
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.555
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.469
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.190
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.462
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.647
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.626
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.626
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.626
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.355
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.648
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.800


2021-05-19 07:14:58,398 - mmdet - INFO - Epoch [10][100/409]	lr: 1.000e-02, eta: 2:41:21, time: 1.687, data_time: 0.044, memory: 21371, loss_cls: 0.4105, loss_bbox: 0.2629, loss_dfl: 0.2085, loss: 0.8818, grad_norm: 3.6983
2021-05-19 07:17:40,843 - mmdet - INFO - Epoch [10][200/409]	lr: 1.000e-02, eta: 2:38:43, time: 1.624, data_time: 0.018, memory: 21371, loss_cls: 0.4188, loss_bbox: 0.2744, loss_dfl: 0.2157, loss: 0.9089, grad_norm: 3.7363
2021-05-19 07:20:23,244 - mmdet - INFO - Epoch [10][300/409]	lr: 1.000e-02, eta: 2:36:06, time: 1.624, data_time: 0.018, memory: 21371, loss_cls: 0.4308, loss_bbox: 0.2638, loss_dfl: 0.2106, loss: 0.9052, grad_norm: 3.7433
